In [257]:
#import dependencies
import os
import pandas as pd

In [258]:
stations_df = pd.read_csv(os.path.join('resources','hawaii_stations.csv'))
meas_df = pd.read_csv(os.path.join('resources','hawaii_measurements.csv'))

In [259]:
#Find out if there are missing values in our data
print(f'Total prcp missing values: {len(meas_df[pd.isnull(meas_df["prcp"])])}')
print(f'Total station missing values: {len(meas_df[pd.isnull(meas_df["station"])])}')
print(f'Total date missing values: {len(meas_df[pd.isnull(meas_df["date"])])}')
print(f'Total tobs missing values: {len(meas_df[pd.isnull(meas_df["tobs"])])}')

Total prcp missing values: 1447
Total station missing values: 0
Total date missing values: 0
Total tobs missing values: 0


#####  We will come up with a dictionary 
```python
{
station_id_1: {jan:median, feb:median, ... }, 
station_id_2: {jan:median, feb:median, ... },
... }
```
##### to fill in missing values. The median is taken over the measurements for all the years recorded

In [244]:
#create new dataframe per station
stations = meas_df['station'].unique()
stations_df = [meas_df[meas_df['station'] == station] for station in stations]
stations_df[0].head(5)

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [245]:
#add a column with month
for station in stations_df:
    station['month'] = station['date'].apply(lambda x: x.split('-')[1])
stations_df[0].head(5)
#add month to the measurements dataframe as well:
meas_df['month'] = meas_df['date'].apply(lambda x: x.split('-')[1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [246]:
station_month_to_mean_prcp = {}
for station_df in stations_df:
    station_id = station_df.iloc[0,0]
    #print(station_id)
    station_mo_gp = station_df.groupby('month')
    median_mo = station_mo_gp.median()
    median_mo_dict = median_mo.loc[:,'prcp'].to_dict()
    #print(median_mo_dict)
    station_month_to_mean_prcp[station_id] = median_mo_dict

In [247]:
#mean precipitaion in November at USC00513117 is 0.299
station_month_to_mean_prcp['USC00513117']['11']


0.029999999999999999

In [248]:
meas_df.head(2)

,station,date,prcp,tobs,month
0,USC00519397,2010-01-01,0.08,65,01
1,USC00519397,2010-01-02,0.00,63,01


In [249]:
len(meas_df[pd.isnull(meas_df["prcp"])])

1447

In [254]:
#iterate over the rows with null values only and use our dictionary to update 'prcp' with the per-staiton, per-month median value
for index, row in meas_df[pd.isnull(meas_df["prcp"])].iterrows():
    stat = row['station']
    month = row['month']
    median_prcp = station_month_to_mean_prcp[stat][month]
    print (stat,month, row['prcp'], median_prcp)
    #WRONG 
    meas_df.iloc[index,2] = median_prcp

USC00519397 01 nan 0.0
USC00519397 01 nan 0.0
USC00519397 02 nan 0.0
USC00519397 02 nan 0.0
USC00519397 03 nan 0.0
USC00519397 03 nan 0.0
USC00519397 05 nan 0.0
USC00519397 07 nan 0.0
USC00519397 11 nan 0.0
USC00519397 11 nan 0.0
USC00519397 12 nan 0.0
USC00519397 01 nan 0.0
USC00519397 02 nan 0.0
USC00519397 03 nan 0.0
USC00519397 06 nan 0.0
USC00519397 10 nan 0.0
USC00519397 06 nan 0.0
USC00519397 06 nan 0.0
USC00519397 07 nan 0.0
USC00519397 08 nan 0.0
USC00519397 08 nan 0.0
USC00519397 12 nan 0.0
USC00519397 12 nan 0.0
USC00519397 01 nan 0.0
USC00519397 01 nan 0.0
USC00519397 07 nan 0.0
USC00519397 01 nan 0.0
USC00519397 01 nan 0.0
USC00519397 05 nan 0.0
USC00519397 05 nan 0.0
USC00519397 05 nan 0.0
USC00519397 08 nan 0.0
USC00519397 08 nan 0.0
USC00519397 03 nan 0.0
USC00519397 03 nan 0.0
USC00519397 08 nan 0.0
USC00519397 08 nan 0.0
USC00519397 09 nan 0.0
USC00519397 09 nan 0.0
USC00513117 02 nan 0.0
USC00513117 09 nan 0.04
USC00513117 08 nan 0.035
USC00513117 09 nan 0.04
USC0051

USC00517948 03 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 04 nan 0.0
USC00517948 05 nan 0.0
USC00517948 05 nan 0.0
USC00517948 05 nan 0.0
USC00517948 05 nan 0.0
USC00517948 05 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 06 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 07 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 08 nan 0.0
USC00517948 09 nan 0.0
USC00517948 09 nan 0.0
USC00517948

USC00517948 10 nan 0.0
USC00517948 10 nan 0.0
USC00517948 10 nan 0.0
USC00517948 10 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 11 nan 0.0
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 12 nan 0.02
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 nan 0.0
USC00517948 01 na

In [255]:
len(meas_df[pd.isnull(meas_df["prcp"])])

0

In [256]:
meas_df.head(50)

,station,date,prcp,tobs,month
0,USC00519397,2010-01-01,0.08,65,01
1,USC00519397,2010-01-02,0.00,63,01
2,USC00519397,2010-01-03,0.00,74,01
3,USC00519397,2010-01-04,0.00,76,01
4,USC00519397,2010-01-06,0.00,73,01
5,USC00519397,2010-01-07,0.06,70,01
6,USC00519397,2010-01-08,0.00,64,01
7,USC00519397,2010-01-09,0.00,68,01
8,USC00519397,2010-01-10,0.00,73,01
9,USC00519397,2010-01-11,0.01,64,01
